In [10]:
import torch
import torch.onnx
import torch.nn as nn

# from maraboupy import Marabou
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
from torchvision import datasets, transforms

In [13]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.,), (1.,))
        ])

test_set = datasets.MNIST('./data', train=False,
                          transform=transform)

In [14]:
cnt = 0

In [18]:
net = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Linear(64, 10),
)
net.load_state_dict(torch.load("./mnist_fc_64x4_adv.model")['state_dict'])
net.to(device)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=64, bias=True)
  (2): ReLU()
  (3): Linear(in_features=64, out_features=64, bias=True)
  (4): ReLU()
  (5): Linear(in_features=64, out_features=64, bias=True)
  (6): ReLU()
  (7): Linear(in_features=64, out_features=64, bias=True)
  (8): ReLU()
  (9): Linear(in_features=64, out_features=10, bias=True)
)

In [21]:
with torch.no_grad():
    cnt = 0
    for i in range(len(test_set)):
        example = test_set[i][0].to(device)
        res = net(example)
        pred = torch.argmax(res)
        gt = test_set[i][1]
        if pred.item() == gt:
            cnt += 1
    print(f"Accuracy: {cnt/len(test_set)}")

Accuracy: 0.7665


In [34]:
train_set = datasets.MNIST('./data', train=True,
                          transform=transform)

In [35]:
normalized_train = torch.empty((60000, 28, 28))

In [36]:
for i in range(60000):
    normalized_train[i] = train_set[i][0]

In [37]:
torch.save(normalized_train, "./dataset/normalized_train.pt")

In [38]:
nt_targets = torch.empty(60000)

In [39]:
for i in range(60000):
    nt_targets[i] = train_set[i][1]

In [40]:
torch.save(nt_targets, "./dataset/nt_targets.pt")